# Import

In [1]:
import time

In [2]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

/home/container_user/.cache/pypoetry/virtualenvs/hansoldeco3-QckXwucX-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import transformers
import accelerate
print(transformers.__version__)
print(accelerate.__version__)

4.49.0
1.4.0


In [4]:
print("CUDA 사용 가능:", torch.cuda.is_available())
print("GPU 개수:", torch.cuda.device_count())
print("사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU 사용 중")

CUDA 사용 가능: True
GPU 개수: 1
사용 중인 GPU: NVIDIA GeForce RTX 3080


# Data Load & Pre-processing

In [5]:
path_train = "/workspace/Storage/hansoldeco3/Data/raw/train.csv"
path_test = "/workspace/Storage/hansoldeco3/Data/raw/test.csv"
sample_idx = "/workspace/Storage/hansoldeco3/Data/sample/sample_v1.csv"

In [6]:
train = pd.read_csv(path_train, encoding = 'utf-8-sig')
test = pd.read_csv(path_test, encoding = 'utf-8-sig')
sample_idx = pd.read_csv(sample_idx)

In [7]:
# val 데이터 선택
test = train[train["ID"].isin(sample_idx["sample_v1"])]
# train 데이터 선택 (val에 포함되지 않은 나머지)
train = train[~train["ID"].isin(sample_idx["sample_v1"])]

In [8]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [9]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [10]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [12]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

print(model.hf_device_map)  # GPU 정상 할당 여부 확인

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.82s/it]


{'': 0}


# Vector store 생성

In [13]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


/tmp/ipykernel_133630/3768745628.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


# RAG chain 생성

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

Device set to use cuda:0
/tmp/ipykernel_133630/3752858109.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


# Inference

In [15]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():

    t = time.time()
    print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    print(row['question'])
    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(
        row['question'],
        # llm_kwargs={"max_new_tokens": 50, "use_cache": True, "do_sample": False}
        )

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)
    print(f"iter {idx} 완료 : {time.time()-t}초")
print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 100

[샘플 1/100] 진행 중...
공사종류 대분류 '토목', 중분류 '도로' 공사 중 공종 대분류 '토목', 중분류 '도로 및 포장공사' 작업에서 사고객체 '건설공구'(중분류: '공구류')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '거푸집 설치를 위해 함마드릴 사용 중 손목이 뒤틀리며 통증'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 0 완료 : 25.45524787902832초

[샘플 2/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설자재'(중분류: '철근')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '철근 작업자가 PIT층에서 철근 절곡기를 이용한 절단작업 중 절단된 철근(약 3cm)이 인중으로 튀면서 살짝 찢어짐'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 1 완료 : 26.643089532852173초

[샘플 3/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '가시설'(중분류: '지지대')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '지정된 이동수단을 사용하지 않고 내려오는 중 미끄러짐'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 2 완료 : 24.63520312309265초

[샘플 4/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설자재'(중분류: '자재')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '젖은 장갑 사용'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 3 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


iter 9 완료 : 26.433717966079712초

[샘플 11/100] 진행 중...
공사종류 대분류 '토목', 중분류 '기타' 공사 중 공종 대분류 '토목', 중분류 '토공사' 작업에서 사고객체 '건설기계'(중분류: '굴착기')와 관련된 사고가 발생했습니다. 작업 프로세스는 '운반작업'이며, 사고 원인은 '굴삭기 운전원 운전 부주의'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 10 완료 : 23.816267013549805초

[샘플 12/100] 진행 중...
공사종류 대분류 '토목', 중분류 '기타' 공사 중 공종 대분류 '토목', 중분류 '토공사' 작업에서 사고객체 '건설기계'(중분류: '굴착기')와 관련된 사고가 발생했습니다. 작업 프로세스는 '굴착작업'이며, 사고 원인은 '굴삭기 하차시 하차용 발판 딛던 중 궤도발판에 묻어있던 젖은 토사를 확인하지 못해 미끄러짐 사고 발생'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 11 완료 : 24.253729820251465초

[샘플 13/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '질병'(중분류: '질병')와 관련된 사고가 발생했습니다. 작업 프로세스는 '기타'이며, 사고 원인은 '사고원인은 해당되는 사항이 없어 관계없는 주원인 선택,  구체적 사고 원인은 경찰조사결과 사인 미상(심정지 추정)'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 12 완료 : 24.324557065963745초

[샘플 14/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철골공사' 작업에서 사고객체 '부재'(중분류: '철골부재')와 관련된 사고가 발생했습니다. 작업 프로세스는 '조립작업'이며, 사고 원인은 'nan'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 13 완료 

In [16]:
pd.DataFrame(test_results, columns = ['answer']).to_csv('./sample_result.csv')

# Submission

In [18]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [20]:
submission = pd.read_csv('/workspace/Storage/hansoldeco3/Data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')